# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
#Comment
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:\\Users\\Owner\\OneDrive\\Desktop\\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,conduaga,8.8620,117.4893,25.30,79,64,1.35,PH,1708125896
1,1,ambalangoda,6.2355,80.0538,24.93,88,97,2.67,LK,1708125896
2,2,port alfred,-33.5906,26.8910,16.21,72,13,0.45,ZA,1708125896
3,3,san patricio,28.0170,-97.5169,18.05,94,100,3.80,US,1708125897
4,4,bethel,41.3712,-73.4140,2.19,50,0,2.57,US,1708125651


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = True, frame_width = 500, frame_height = 400, size = city_data_df["Humidity"]/2)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] >= 16) & (city_data_df["Max Temp"] <= 20) & (city_data_df["Humidity"] < 70) & (city_data_df["Wind Speed"] <= 3)]
new_city_data_df

# Drop any rows with null values
new_city_data_df = new_city_data_df.dropna(subset = ["Max Temp", "Wind Speed"])


# Display sample data
new_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,papatowai,-46.5619,169.4708,18.46,55,95,0.85,NZ,1708125897
94,94,hermanus,-34.4187,19.2345,17.05,64,39,1.12,ZA,1708125943
112,112,hassi messaoud,31.6804,6.0729,17.92,39,100,1.54,DZ,1708125948
140,140,itoman,26.1247,127.6694,19.01,68,75,1.54,JP,1708125958
243,243,khamharia,20.9833,82.2667,19.19,57,0,1.23,IN,1708125997
279,279,mount gambier,-37.8333,140.7667,19.07,60,64,2.93,AU,1708126007
308,308,dunedin,-45.8742,170.5036,17.76,62,79,2.28,NZ,1708126017
323,323,ceerigaabo,10.6162,47.3680,16.04,53,14,1.44,SO,1708126022
330,330,jaisalmer,26.9147,70.9181,18.75,15,2,2.80,IN,1708126024
419,419,turbat,26.0023,63.0440,19.92,48,100,1.18,PK,1708126052


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city_data_df.copy(deep = True)
hotel_df = hotel_df[["City_ID", "City", "Country", "Lat", "Lng", "Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
5,5,papatowai,NZ,-46.5619,169.4708,55,
94,94,hermanus,ZA,-34.4187,19.2345,64,
112,112,hassi messaoud,DZ,31.6804,6.0729,39,
140,140,itoman,JP,26.1247,127.6694,68,
243,243,khamharia,IN,20.9833,82.2667,57,
279,279,mount gambier,AU,-37.8333,140.7667,60,
308,308,dunedin,NZ,-45.8742,170.5036,62,
323,323,ceerigaabo,SO,10.6162,47.3680,53,
330,330,jaisalmer,IN,26.9147,70.9181,15,
419,419,turbat,PK,26.0023,63.0440,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000


params = {
    "limit" : 1,
    "apiKey" : geoapify_key,
    "categories" : "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
papatowai - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
hassi messaoud - nearest hotel: AMC
itoman - nearest hotel: No hotel found
khamharia - nearest hotel: No hotel found
mount gambier - nearest hotel: The Old Mount Gambier Gaol
dunedin - nearest hotel: Scenic Hotel Dunedin City
ceerigaabo - nearest hotel: Sanaag Hotel
jaisalmer - nearest hotel: Jeet Mahal
turbat - nearest hotel: No hotel found
zalingei - nearest hotel: No hotel found
greymouth - nearest hotel: Recreation Hotel


,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
5,5,papatowai,NZ,-46.5619,169.4708,55,No hotel found
94,94,hermanus,ZA,-34.4187,19.2345,64,Aloe guest house
112,112,hassi messaoud,DZ,31.6804,6.0729,39,AMC
140,140,itoman,JP,26.1247,127.6694,68,No hotel found
243,243,khamharia,IN,20.9833,82.2667,57,No hotel found
279,279,mount gambier,AU,-37.8333,140.7667,60,The Old Mount Gambier Gaol
308,308,dunedin,NZ,-45.8742,170.5036,62,Scenic Hotel Dunedin City
323,323,ceerigaabo,SO,10.6162,47.3680,53,Sanaag Hotel
330,330,jaisalmer,IN,26.9147,70.9181,15,Jeet Mahal
419,419,turbat,PK,26.0023,63.0440,48,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = True, frame_width = 500, frame_height = 400, size = (hotel_df["Humidity"]/2), hover_cols = ["Country","Hotel Name"])

# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_size,Country,Hotel Name)